## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,29.86,86,75,17.27,light snow
1,1,Coihaique,CL,-45.5752,-72.0662,43.41,75,75,4.61,light rain
2,2,Hilo,US,19.7297,-155.0900,73.13,90,90,5.75,light rain
3,3,Bethel,US,41.3712,-73.4140,84.38,71,1,10.36,clear sky
4,4,Adrar,MR,20.5022,-10.0711,104.49,18,3,13.60,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
chosen_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
chosen_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hilo,US,19.7297,-155.0900,73.13,90,90,5.75,light rain
3,3,Bethel,US,41.3712,-73.4140,84.38,71,1,10.36,clear sky
5,5,Esperance,AU,-33.8667,121.9000,58.84,83,100,18.39,light rain
7,7,Busselton,AU,-33.6500,115.3333,59.09,92,100,14.94,light rain
8,8,Granville,AU,-33.8333,151.0167,54.30,71,75,1.01,broken clouds
9,9,Vaini,TO,-21.2000,-175.2000,66.36,82,20,6.91,few clouds
10,10,Northam,GB,51.0333,-4.2167,63.39,88,62,23.06,broken clouds
11,11,East London,ZA,-33.0153,27.9116,63.54,77,20,9.22,few clouds
12,12,Flinders,AU,-34.5833,150.8552,54.21,75,98,1.01,overcast clouds
14,14,Klaksvik,FO,62.2266,-6.5890,67.12,88,100,21.05,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
chosen_cities_df.count()

City_ID                526
City                   526
Country                521
Lat                    526
Lng                    526
Max Temp               526
Humidity               526
Cloudiness             526
Wind Speed             526
Current Description    526
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = chosen_cities_df.dropna()
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hilo,US,19.7297,-155.0900,73.13,90,90,5.75,light rain
3,3,Bethel,US,41.3712,-73.4140,84.38,71,1,10.36,clear sky
5,5,Esperance,AU,-33.8667,121.9000,58.84,83,100,18.39,light rain
7,7,Busselton,AU,-33.6500,115.3333,59.09,92,100,14.94,light rain
8,8,Granville,AU,-33.8333,151.0167,54.30,71,75,1.01,broken clouds
...,...,...,...,...,...,...,...,...,...,...
712,712,Zhangye,CN,38.9342,100.4517,67.55,48,79,10.85,broken clouds
713,713,Kyren,RU,51.6828,102.1408,58.46,88,96,2.13,overcast clouds
714,714,Teguise,ES,29.0605,-13.5640,77.22,57,20,18.41,few clouds
715,715,Raga,SS,8.4596,25.6780,75.52,87,93,4.07,light rain


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hilo,US,73.13,light rain,19.7297,-155.0900,
3,Bethel,US,84.38,clear sky,41.3712,-73.4140,
5,Esperance,AU,58.84,light rain,-33.8667,121.9000,
7,Busselton,AU,59.09,light rain,-33.6500,115.3333,
8,Granville,AU,54.30,broken clouds,-33.8333,151.0167,
9,Vaini,TO,66.36,few clouds,-21.2000,-175.2000,
10,Northam,GB,63.39,broken clouds,51.0333,-4.2167,
11,East London,ZA,63.54,few clouds,-33.0153,27.9116,
12,Flinders,AU,54.21,overcast clouds,-34.5833,150.8552,
14,Klaksvik,FO,67.12,overcast clouds,62.2266,-6.5890,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df['Hotel Name'].isnull().sum()

51

In [11]:
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   470
Country                470
Max Temp               470
Current Description    470
Lat                    470
Lng                    470
Hotel Name             470
dtype: int64

In [12]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hilo,US,73.13,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
3,Bethel,US,84.38,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
5,Esperance,AU,58.84,light rain,-33.8667,121.9000,Hospitality Esperance
7,Busselton,AU,59.09,light rain,-33.6500,115.3333,Observatory Guest House
8,Granville,AU,54.30,broken clouds,-33.8333,151.0167,Mercure Sydney Parramatta
...,...,...,...,...,...,...,...
711,Hope,US,84.25,clear sky,33.6671,-93.5916,Best Western Hope
712,Zhangye,CN,67.55,broken clouds,38.9342,100.4517,Zhangye Lvzhou Holiday Inn
713,Kyren,RU,58.46,overcast clouds,51.6828,102.1408,"Energetik, Pansionat Ooo ""Grand Baykal"""
714,Teguise,ES,77.22,few clouds,29.0605,-13.5640,B&B La Casona de Nazaret


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} with Max Temp of: {Max Temp}°</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))